## Setup

In [1]:
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")

import matplotlib.pyplot as plt
import numpy as np

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPUs will likely run quickly with dtype policy mixed_float16 as they all have compute capability of at least 7.0


In [2]:
try: 
    tpu = None
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError: 
    strategy = tf.distribute.MirroredStrategy() 

print("Number of accelerators: ", strategy.num_replicas_in_sync)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Number of accelerators:  4


## Define hyperparameters

In [3]:
TARGET_SIZE = (150, 150)

AUTO = tf.data.AUTOTUNE
BATCH_SIZE = 128 * strategy.num_replicas_in_sync
EPOCHS = 10

## Prepare data loaders

In [4]:
train_ds, validation_ds, test_ds = tfds.load(
    "cats_vs_dogs",
    # Reserve 10% for validation and 10% for test
    split=["train[:90%]", "train[90%:95%]", "train[95%:]"],
    as_supervised=True,  
)

In [5]:
def preprocess_dataset(image, label):
    image = tf.image.resize(image, (TARGET_SIZE[0], TARGET_SIZE[1]))
    label = tf.one_hot(label, depth=2)
    return (image, label)

In [6]:
train_ds = (
    train_ds
    .shuffle(BATCH_SIZE * 100)
    .map(preprocess_dataset, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)
validation_ds = (
    validation_ds
    .map(preprocess_dataset, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)
test_ds = (
    test_ds
    .map(preprocess_dataset, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

## Build, train, and evaluate model

In [7]:
def get_model():
    input_shape = (TARGET_SIZE[0], TARGET_SIZE[0], 3)
    backbone = tf.keras.applications.DenseNet121(weights=None, include_top=True, classes=2, 
                                              input_shape=input_shape)
    backbone.trainable = True
    
    inputs = layers.Input(input_shape)
    x = layers.experimental.preprocessing.Rescaling(scale=1./127.5, offset=-1)(inputs)
    outputs = backbone(x)
    outputs = layers.Activation("linear", dtype="float32")(outputs)

    return tf.keras.Model(inputs, outputs)

In [8]:
# We will use this initial random model to train both the models
# for reproducibility
initial_model = get_model()
initial_model.save("initial_model")

INFO:tensorflow:Assets written to: initial_model/assets


INFO:tensorflow:Assets written to: initial_model/assets


The model weights that were initially used can be retrieved from [here](https://github.com/sayakpaul/Learnable-Image-Resizing/releases/tag/v1.0.0):

```shell
!wget -q https://github.com/sayakpaul/Learnable-Image-Resizing/releases/download/v1.0.0/models.tar.gz
!tar xf models.tar.gz
```

In [9]:
with strategy.scope():
    model = tf.keras.models.load_model("initial_model")
    model.compile(loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
                optimizer="sgd",
                metrics=["accuracy"])
model.fit(train_ds,
          validation_data=validation_ds,
          epochs=EPOCHS)

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


...

Epoch 1/10
INFO:tensorflow:batch_all_reduce: 364 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 364 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 364 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 364 all-reduces with algorithm = nccl, num_packs = 1


41/41 [==============================] - 187s 511ms/step - loss: 0.6950 - accuracy: 0.5813 - val_loss: 0.6946 - val_accuracy: 0.4953
Epoch 2/10
41/41 [==============================] - 13s 230ms/step - loss: 0.6364 - accuracy: 0.6545 - val_loss: 0.6962 - val_accuracy: 0.4944
Epoch 3/10
41/41 [==============================] - 12s 229ms/step - loss: 0.6226 - accuracy: 0.6717 - val_loss: 0.6949 - val_accuracy: 0.4918
Epoch 4/10
41/41 [==============================] - 12s 229ms/step - loss: 0.6099 - accuracy: 0.6923 - val_loss: 0.7030 - val_accuracy: 0.4953
Epoch 5/10
41/41 [==============================] - 13s 232ms/step - loss: 0.6011 - accuracy: 0.7000 - val_loss: 0.7317 - val_accuracy: 0.4918
Epoch 6/10
41/41 [==============================] - 12s 227ms/step - loss: 0.5826 - accuracy: 0.7230 - val_loss: 0.7702 - val_accuracy: 0.4961
Epoch 7/10
41/41 [==============================] - 12s 227ms/step - loss: 0.5744 - accuracy: 0.7302 - val_loss: 0.7835 - val_accuracy: 0.5064
Epoch 8/1

In [10]:
with strategy.scope():
    _, test_acc = model.evaluate(test_ds)
print("Test accuracy: {:.2f}%".format(test_acc * 100))

model.save("standard_densenet_model")

3/3 [==============================] - 1s 124ms/step - loss: 0.7059 - accuracy: 0.6019
Test accuracy: 60.19%
INFO:tensorflow:Assets written to: standard_densenet_model/assets


INFO:tensorflow:Assets written to: standard_densenet_model/assets
